In [1]:
import requests
import re

#selenium imports
import bs4
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# to prevent Just-Eat from blocking us... not needed with selenium though.
user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.37"

# no need to set user agent, cause thats already done through webdriver.Firefox()
browser = webdriver.Firefox()
browser.get('https://www.just-eat.dk/area/2800-kgs--lyngby/')
browser.implicitly_wait(3)

# accept cookies
WebDriverWait(browser,20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div/div[2]/button[1]'))).click()

page_source = browser.page_source

soup = bs4.BeautifulSoup(page_source, 'html.parser')

event_cells = soup.findAll('h3', attrs= {'class': 'listing-item-title'})
pizza_checker = re.compile(r'([\w ]+)*(Pizza|Pizzeria|Pizzaria)([\w ]+)*')
pizza_list = []
for e in event_cells:
    restaurant_with_pizza = pizza_checker.match(str(e.text))
    if restaurant_with_pizza is not None:
        pizza_list.append(restaurant_with_pizza.group())

print(pizza_list)

# new
soup = bs4.BeautifulSoup(page_source, 'html.parser')
restaurant_links = []
for a in soup.select('a'):
    if (a.get('href') and a.get('href').startswith('/restaurants')):
        link = 'https://www.just-eat.dk' + a.get('href')
        restaurant_links.append(link)
#print(restaurant_links)

margherita_priser = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36 OPR/74.0.3911.218'
}
def get_margherita_price(url):
    r = requests.get(url, headers = headers)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    products = soup.findAll('div', attrs= {'class': 'menu-product'})
    margherita_reg = re.compile(r'Margherita')
    price_reg = re.compile(r'(\d{2})')
    for p in products:
        if (margherita_reg.search(p.getText()) is not None):
            margherita_priser[url] = price_reg.search(p.getText()).group()
    
for link in restaurant_links:
    get_margherita_price(link)

print("Price:", margherita_priser)

['Jernbane Pizza Durum Kebab House', 'Pizza Time', 'La Luna Pizzaria', 'Alanya Pizza ', 'Il Mondo Pizzaria', 'Eda Pizzaria og Burgerhouse', 'Mama Rosa Pizzeria', 'Pops Pizza', 'Canaria Pizzaria og Grill', 'Saras Pizzaria ', 'Madhuset Pizza ', 'Miss Pizza', 'Nybrogård Pizza ', 'La Casa Pizza', 'Zetuna Pizzaria ', 'Søndergaard Park Pizzaria', 'La Rosa Pizzaria ', 'Værebro Pizza', 'Pizzahuset', 'Simona Pizzeria ', 'Buddinge Pizza', 'Itzi Pitzi Pizza', 'Pizza 2860', 'Esperia Pizza Bar', 'Barletta Pizzeria', 'La Tinos Pizza', 'Corona Pizza ', 'Miezio Pizza', 'Pizza Perfetto']
Price: {'https://www.just-eat.dk/restaurants-jernbanekebab/menu': '74', 'https://www.just-eat.dk/restaurants-gepettos/menu': '56', 'https://www.just-eat.dk/restaurants-dominos-lyngby-/menu': '20', 'https://www.just-eat.dk/restaurants-mevlana-2200/menu': '79', 'https://www.just-eat.dk/restaurants-pizzatime2400/menu': '45', 'https://www.just-eat.dk/restaurants-laluna/menu': '50', 'https://www.just-eat.dk/restaurants-alan